In [1]:
! pip install transformers datasets torch evaluate arabert wandb

In [2]:
! wandb login 80865993e8bb9bc09d51465ad41acdec4e354ee1


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [3]:
! pip install gdown
import gdown
gdown.download("https://drive.google.com/uc?id=1wEj32Ie9jnTDx3-iw8hBLyP3UZRthgtY", "all.txt", quiet=False)


Downloading...
From: https://drive.google.com/uc?id=1wEj32Ie9jnTDx3-iw8hBLyP3UZRthgtY
To: /kaggle/working/all.txt
100%|██████████| 329k/329k [00:00<00:00, 81.8MB/s]


'all.txt'

In [4]:
import wandb
# Step 1: Initialize Weights & Biases
wandb.init(project="allama-qa-finetuning", name="AraT5-run")


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: abdoashraf952 (abdoashraf952-google). Use `wandb login --relogin` to force relogin


In [5]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, Trainer, TrainingArguments
from datasets import load_dataset
import numpy as np
import evaluate  # Hugging Face `evaluate` library


model_name = "UBC-NLP/AraT5v2-base-1024"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [6]:

dataset = {"input_text": [], "target_text": []}

with open("/kaggle/working/all.txt", "r") as f:
  lines = f.readlines()
  for i in range(0, len(lines), 2):  # Step by 2 to get input and target
    dataset["input_text"].append(lines[i].strip())  
    dataset["target_text"].append(lines[i+1].strip())

# تحويلها إلى Dataset خاص بـ Hugging Face
from datasets import Dataset
train_dataset = Dataset.from_dict(dataset)

# تحويل النصوص إلى Tokens
def preprocess_function(examples):
    inputs = tokenizer(examples["input_text"], padding="max_length", truncation=True, max_length=128)
    targets = tokenizer(examples["target_text"], padding="max_length", truncation=True, max_length=128)
    inputs["labels"] = targets["input_ids"]
    return inputs

accuracy_metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)  # Get predicted class
    return accuracy_metric.compute(predictions=predictions, references=labels)


train_dataset = train_dataset.map(preprocess_function, batched=True)


Map:   0%|          | 0/2633 [00:00<?, ? examples/s]

In [7]:
# Assume train_dataset is already loaded
test_size = 0.1  # Allocate 10% for testing
train_size = len(train_dataset) - int(test_size * len(train_dataset))

# Split into train and test
train_dataset, test_data = train_dataset.train_test_split(
    train_size=train_size, test_size=int(test_size * len(train_dataset)), seed=42).values()

# Now split the remaining train dataset into training and validation
train_size = int(0.8 * len(train_dataset))
eval_size = len(train_dataset) - train_size

train_data, eval_data = train_dataset.train_test_split(
    train_size=train_size, test_size=eval_size, seed=42).values()

In [8]:
training_args = TrainingArguments(
    output_dir="./AraT5_finetuned",
    evaluation_strategy="epoch",
    logging_strategy="epoch",  # تصحيح التحذير
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=50,  
    save_steps=5000,
    save_total_limit=1,
    logging_dir="./logs",
    logging_steps=500,
    load_best_model_at_end=True,  # تحميل أفضل نموذج
    metric_for_best_model="loss",  # تتبع أقل قيمة لـ loss
    greater_is_better=False,
    save_strategy="epoch",  
    report_to="wandb",
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [9]:

from transformers import EarlyStoppingCallback

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=eval_data,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=5)],  # إيقاف التدريب بعد 5 إيبكات بدون تحسن
)


In [10]:

# بدء Fine-Tuning
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,15.474500,7.472571
2,3.220600,0.725972
3,0.735300,0.563972
4,0.606500,0.513926
5,0.519800,0.335374
6,0.358500,0.256219
7,0.289100,0.227024
8,0.255000,0.213367
9,0.232400,0.201781
10,0.215800,0.194670


/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked t

TrainOutput(global_step=3332, training_loss=0.8768204191581113, metrics={'train_runtime': 4733.3276, 'train_samples_per_second': 20.028, 'train_steps_per_second': 1.257, 'total_flos': 1.1532985555746816e+16, 'train_loss': 0.8768204191581113, 'epoch': 28.0})

In [11]:
import torch
torch.cuda.empty_cache()

In [12]:
model.save_pretrained("./AraT5_finetuned_model")
tokenizer.save_pretrained("./AraT5_finetuned_model")

('./AraT5_finetuned_model/tokenizer_config.json',
 './AraT5_finetuned_model/special_tokens_map.json',
 './AraT5_finetuned_model/spiece.model',
 './AraT5_finetuned_model/added_tokens.json',
 './AraT5_finetuned_model/tokenizer.json')

In [13]:
validation_metrics = trainer.evaluate()
print(validation_metrics)


/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


{'eval_loss': 0.17167551815509796, 'eval_runtime': 14.2897, 'eval_samples_per_second': 33.171, 'eval_steps_per_second': 2.099, 'epoch': 28.0}


In [14]:
! zip -r /kaggle/working/AraT5_finetuned_model.zip /kaggle/working/AraT5_finetuned_model

  adding: kaggle/working/AraT5_finetuned_model/ (stored 0%)
  adding: kaggle/working/AraT5_finetuned_model/config.json (deflated 47%)
  adding: kaggle/working/AraT5_finetuned_model/tokenizer.json (deflated 84%)
  adding: kaggle/working/AraT5_finetuned_model/model.safetensors (deflated 18%)
  adding: kaggle/working/AraT5_finetuned_model/special_tokens_map.json (deflated 85%)
  adding: kaggle/working/AraT5_finetuned_model/tokenizer_config.json (deflated 94%)
  adding: kaggle/working/AraT5_finetuned_model/spiece.model (deflated 55%)
  adding: kaggle/working/AraT5_finetuned_model/generation_config.json (deflated 29%)


In [15]:
!ls /kaggle/working/

all.txt		 AraT5_finetuned_model	    wandb
AraT5_finetuned  AraT5_finetuned_model.zip


In [16]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# تحميل النموذج المحفوظ
model_path = "/kaggle/working/AraT5_finetuned_model"
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)


In [17]:

# اختبار النموذج مع نص جديد
input_text = "امتى البيت اتسرق"
inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)

# توليد النص باستخدام النموذج المدرب
output_tokens = model.generate(**inputs)
output_text = tokenizer.decode(output_tokens[0], skip_special_tokens=True)

print( output_text)


امبارح الساعة 5 العصر
